# RNNを用いた言語モデル

RNNを用いて言語モデルを実装する。

前章では、深層学習を活用してある単語から次の単語を予測するモデルを作成した。しかし、これでは全体の文脈全体を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。

RNNを用いることで、文脈を考慮した予測が可能になる。  
本章ではRNNの構造について学び、RNNを用いた言語モデルを実装する。

In [1]:
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 学習データの用意

In [3]:
n_data = 1000

wiki40b

In [4]:
textfile = 'data/jawiki.txt'
with open(textfile) as f:
    data = f.readlines()

In [5]:
data = data[:n_data]
textfile = f'data/jawiki_{n_data}.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence)

print('num of data:', len(data))
data[:3] # examples

num of data: 1000


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。\n',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。\n']


---

## RNN

*Recurrent Neural Network*

再帰型ニューラルネットワーク。  
再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。

RNNはある時間$t$の入力$x_t$に対して以下のような演算を行い、出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

演算の内部で前の時間の出力値$h_{t-1}$を参照していることが分かる。  
演算内容はシンプルに捉えることができて、入力$x_t$と前の時間の出力$h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数$\mathrm{tanh}$に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。

実装してみるとこんな感じ。

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size: int, output_size: int):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_output = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        """
            x: (batch_size, input_size)
            h: (batch_size, output_size)
        """
        z = self.fc_input(x) + self.fc_output(h)
        h = F.tanh(z)
        return h # (batch_size, output_size)

### 推論

推論の流れを見てみる。

まず適当なパラメータで出力の初期値$h_0$と初めの入力$x_1$を定義する。$h_0$は0ベクトルで良い。

In [7]:
batch_size, input_size, hidden_size = 2, 3, 4
rnn = RNN(input_size, hidden_size)

x1 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, hidden_size)

h1 = rnn(x1, h0)
h1

tensor([[ 0.3388,  0.5429,  0.7555, -0.2220],
        [ 0.2134, -0.8904, -0.6986,  0.1338]], grad_fn=<TanhBackward0>)

この$h_1$を参照して、次の時間の入力$x_2$に対する出力$h_2$を決定する。

In [8]:
x2 = torch.randn(batch_size, input_size)
h2 = rnn(x2, h1)
h2

tensor([[ 0.2392, -0.8614, -0.7272,  0.5622],
        [ 0.2182, -0.4587, -0.2178,  0.1083]], grad_fn=<TanhBackward0>)

これがRNNの推論の流れである。

### PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

#### `torch.nn.RNNCell`

RNNの一つの時間の演算を行うクラス。↑で実装したものと同じ。  
https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html

In [9]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, '\n')

Parameter containing:
tensor([[ 0.0409,  0.2221,  0.1491],
        [-0.4196,  0.3536, -0.4554],
        [-0.2124,  0.0137,  0.2680],
        [ 0.4148, -0.1994,  0.1721]], requires_grad=True) 

Parameter containing:
tensor([[-0.4359,  0.1736,  0.1761,  0.0394],
        [ 0.2472, -0.1803, -0.3639, -0.0584],
        [-0.3944,  0.0721,  0.0702,  0.4661],
        [-0.4844, -0.2130,  0.2387,  0.0106]], requires_grad=True) 

Parameter containing:
tensor([-0.3562, -0.2456,  0.4223,  0.4898], requires_grad=True) 

Parameter containing:
tensor([ 0.2282,  0.4587, -0.4084,  0.1137], requires_grad=True) 



ちゃんとパラメータが4つあるね。

推論も同じ。

In [10]:
x1 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, hidden_size)
rnn(x1, h0)

tensor([[-0.3310,  0.7138, -0.2357,  0.2534],
        [-0.4185,  0.6526, -0.7238,  0.6171]], grad_fn=<TanhBackward0>)

ちなみに$h$は入力しなくてもいい。その場合は勝手に0ベクトルになる。

In [11]:
rnn(x1, h0)

tensor([[-0.3310,  0.7138, -0.2357,  0.2534],
        [-0.4185,  0.6526, -0.7238,  0.6171]], grad_fn=<TanhBackward0>)

#### `torch.nn.RNN`

全ての時間の演算をまとめて行うクラス。  
https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [12]:
batch_size, input_size, hidden_size, seq_length = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_length, input_size)
hs, h = rnn(x)
hs.shape

torch.Size([2, 5, 4])

全ての時間の隠れ状態が出力される。

`h`は最後の隠れ状態。

In [13]:
hs[:, -1] == h

tensor([[[True, True, True, True],
         [True, True, True, True]]])

In [14]:
h.shape

torch.Size([1, 2, 4])

初めに次元が1つ追加されるのは仕様。


---

## 言語モデルの構築

RNNを用いた言語モデル構築の流れを見てみる。

基本的に、RNNそのものを1つのモデルとして扱うことはない。RNNはNNの中の1つの層として扱う。  
なお、RNNが組み込まれたモデルをRNNと呼ぶこともある。混乱を避けるために、本章の以降では層としてのRNNはRNN層と呼ぶこととする（モデルとしてのRNNはそのままRNNと呼ぶ）。

ある時間$t$におけるRNN層は、前の層からの出力$z_t$と前の時間のRNN層の出力$h_{t-1}$を受け取り、$h_t$を出力する。  
$h_t$は外から直接観測されることがないため、**隠れ状態**（や潜在変数）と表現される。変数名に$h$を使っている理由がそこにある（隠れる=hide）。

RNNは可変長のデータを扱うことが出来るため、色々な形態の入出力を扱える。  
RNNの入出力は単数か複数かで分け、その組み合わせによって次のように分類される。

- one-to-one
- one-to-many
- many-to-one
- many-to-many

例えば、文章から感情を予測する様な分類モデルはmany-to-oneとなる。  
一方で、初めの単語を入れて以降の文章を生成するモデルはone-to-manyに当たる。作ろうとしている言語モデルはこれ。

扱うデータがこれらのどこに該当するかでRNNの構造が変わってくる。

例えばmany-to-oneの場合は、RNN層から出てきた最後の隠れ状態を次の層に渡せばいい。

In [15]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        """
        x: (batch_size, seq_length, input_size)
        """
        _, h = self.rnn(x) # (1, batch_size, hidden_size)
        y = self.fc(h[0]) # (batch_size, input_size)
        return y

batch_size, seq_len, input_size, hidden_size = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(batch_size, seq_len, input_size)
y = rnn(x)
y.shape

torch.Size([2, 4])

では言語モデルを作っていこう。  
one-to-manyの場合は再帰的な処理が必要になるので、隠れ状態の入出力ができるようにする必要がある。また入出力のサイズは語彙の数と同じ。

In [16]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (1, batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_size)
        hs, h = self.rnn(x)
            # hs: (batch_size, seq_length, hidden_size)
            # h: (1, batch_size, hidden_size)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, h

再帰させるのであれば隠れ状態は常に1つなので、こちらも`hs`を使う必要はないように思えるが、学習時はmany-to-manyとして扱うことが多い（詳細は次節）ので、全ての隠れ状態`hs`を次の層に渡すようにする。


---

## BPTT

*Back Propagation Through Time*

時間を跨いだ逆伝播。

RNNでは、ある時間$t$における演算で、その時間の入力$x_t$以外に、隠れ状態（前のRNN層からの出力）$h_{t-1}$を参照する。  
これは$x_t$のみを参照する通常のNNとは大きく異なる点である。

言語モデルの様な、各時間で確率分布からのサンプリングを挟むようなモデルでは、この隠れ状態が重要な役割を果たす。それは逆伝播が時間を跨いで繋がる点である。これをBPTTと呼ぶ。

BPTTによって時間を遡って勾配を届けられるようになり、長期的な視点で誤差が減るように学習できる。その際、勾配の伝達を担う隠れ状態には、長期的な文脈の情報が載るようになることが期待される。

ちなみに、サンプリングを挟まない場合は普通に逆伝播が届く。モデルの出力値をそのまま入力する場合とか。

あとは、サンプリングといっても、出力されたデータの一部を次に入力するような場合も普通に逆伝播が届く。

### Truncated BPTT

BPTTで細かく時間を区切る手法。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなる。ただ、実はRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

実装は以下のようになる。

```python
trunc_len = 64 # 区切る長さ

for x, t in dataloader:
    h = None # 隠れ状態の初期化

    # Truncated BPTT
    for i in range(0, x.shape[1], trunc_len):
        x_batch = x[:, i:i+trunc_len].to(device)
        t_batch = t[:, i:i+trunc_len].to(device)

        optimizer.zero_grad()
        y, h = model(x_batch, h)
        loss = loss_fn(y, t_batch)
        loss.backward()
        optimizer.step()

        h = h.detach() # 計算グラフから切り離す
```

あくまでも区切るのは逆伝播だけなので、順伝播時の隠れ状態はちゃんと保持される。


---

## 学習データの作成

入力と出力のペアを作成する。  
では、どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。  
これは普通に文章を教師強制で学習させているだけとも言える。

In [17]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __getitem__(self, idx):
        text = self.data[idx]
        in_text = text[:-1] # 入力単語列
        out_text = text[1:] # 入力を1つずつずらした単語列
        return in_text, out_text

    def __len__(self):
        return self.n_data


---

## ミニバッチ学習

RNNを用いた言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [18]:
from torch.nn.utils.rnn import pad_sequence

In [19]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [ ]:
pad_id = 3
vocab_size = 8000
tokenizer_prefix = f'models/tokenizer_jawiki_{n_data}'

spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

これでトークン化。

In [21]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model')
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()

data_ids = sp.encode(data)
for ids in data_ids:
    ids.insert(0, bos_id)
    ids.append(eos_id)

n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[1, 12, 19, 6255, 55, 1058, 59, 1686, 80, 123]

そんで、バッチ内データが揃う用にDataLoaderを作成する。

In [22]:
def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 32
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

sample_x, sample_y = next(iter(train_loader))
print('input batch shape:', sample_x.shape)
sample_x[0], sample_y[0] # example

input batch shape: torch.Size([32, 2071])


(tensor([  1,  12, 573,  ...,   3,   3,   3]),
 tensor([ 12, 573,   7,  ...,   3,   3,   3]))

これで学習データはOK。

最後に、学習時に使う交差エントロピーでパディング用のトークンを無視するように定義すれば完了。

In [23]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

では、これまでの内容をまとめて言語モデルを学習させていこう。  

In [24]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss


def eval_model(model, trunc_len=100):
    model.eval()
    losses = []
    with torch.no_grad():
        for x, t in test_loader:
            h = None
            for i in range(0, x.shape[1], trunc_len):
                x_batch = x[:, i:i+trunc_len].to(device)
                t_batch = t[:, i:i+trunc_len].to(device)
                y, h = model(x_batch, h)
                loss = loss_fn(y, t_batch)
                losses.append(loss.item())
    loss = sum(losses) / len(losses)
    return loss


def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    # trunc_len: 区切る長さ

    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            h = None

            # Truncated BPTT
            for i in range(0, x.shape[1], trunc_len):
                x_batch = x[:, i:i+trunc_len].to(device) # (batch_size, trunc_len, n_vocab)
                t_batch = t[:, i:i+trunc_len].to(device) # (batch_size, trunc_len)

                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = loss_fn(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 計算グラフから切り離す
            prog.update()

        if prog.now_epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}', no_step=True)
        prog.memo()

In [25]:
n_vocab = len(sp)
embed_size = 512
hidden_size = 512
model = LanguageModel(n_vocab, embed_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [26]:
train(model, optimizer, trunc_len=30, n_epochs=100, prog_unit=10)

   1-10/100: ############################## 100% [00:01:46.23] loss train: 5.52331, test: 4.04591 
  11-20/100: ############################## 100% [00:01:47.31] loss train: 3.63529, test: 2.62240 
  21-30/100: ############################## 100% [00:01:46.06] loss train: 2.52664, test: 1.82859 
  31-40/100: ############################## 100% [00:01:46.20] loss train: 1.73804, test: 1.37252 
  41-50/100: ############################## 100% [00:01:43.92] loss train: 1.21597, test: 1.09178 
  51-60/100: ############################## 100% [00:01:47.07] loss train: 0.86059, test: 0.94294 
  61-70/100: ############################## 100% [00:01:47.34] loss train: 0.61748, test: 0.87470 
  71-80/100: ############################## 100% [00:01:44.51] loss train: 0.46739, test: 0.82061 
  81-90/100: ############################## 100% [00:01:47.04] loss train: 0.35708, test: 0.79738 
 91-100/100: ############################## 100% [00:01:46.68] loss train: 0.28544, test: 0.79229 



---

## 文章生成

学習したモデルを使って文章を生成する。  
モデルが出力した単語を次の入力とし、再帰的に出力を得る。

In [27]:
def token_sampling(y):
    y.squeeze_(0)
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()

    # 隠れ状態の初期値
    token_ids = sp.encode(start)
    token_ids.insert(0, bos_id)
    x = torch.tensor(token_ids, device=device)
    y, h = model(x)
    next_token = token_sampling(y)
    token_ids.append(next_token)

    # 続きの文章生成
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device)
        y, h = model(x, h)
        next_token = token_sampling(y)
        token_ids.append(next_token)

    sentence = sp.decode(token_ids)
    return sentence

In [28]:
for _ in range(5):
    print(generate_sentence(model, max_len=50))

Offrotはワイド昔はそれぞれの画面の起源(An』において最も有効性の要求どのように自己最高300WPSOIFI参照)。飛びながらも司祭アリウス」が決定だとは
渡良瀬川の判定勝ちをつけるは、コンスタンティヌス1世論んだ。聖護院本山派の受け入れ替えて将軍にマクベスから、それを支えた幼少し、文書を主に得点面の「GTOPホールでの元の末
&CCSといった箱庭にあたる。
e&ランガスと競争以降、札1番かな権利やカーラム(ニカイア派への対置のための標準的バンドの職員の人も消滅亡く何度ものマエスとは無髄線維
異教え、同一の伝要求を全アルバムとなるところ」展覧会に代表となる。1957日、最も有ると鞘する艦隊が過によって近衛長官(ドナウ川の松島スと安全を要求を傷を


マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減ったと思われる。